In [6]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [11]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 903.1 kB/s eta 0:00:00


In [7]:
from datasets import load_dataset

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 78977
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 8776
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21939
    })
})

In [13]:
import tensorflow as tf
import torch

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss
from sklearn.preprocessing import MultiLabelBinarizer

import pandas as pd, numpy as np, random, time, datetime
from tqdm import tqdm

import csv, os

In [14]:
train = load_dataset('jeanlee/kmhas_korean_hate_speech', split='train')
validation = load_dataset('jeanlee/kmhas_korean_hate_speech', split='validation')
test = load_dataset('jeanlee/kmhas_korean_hate_speech', split='test')

In [15]:
train_sentences = list(map(lambda x: '[CLS] '+str(x)+' [SEP]', train['text']))
validation_sentences = list(map(lambda x: '[CLS] '+str(x)+' [SEP]', validation['text']))
test_sentences = list(map(lambda x: '[CLS] '+str(x)+' [SEP]', test['text']))

In [16]:
train_sentences

['[CLS] "자한당틀딱들.. 악플질 고만해라." [SEP]',
 '[CLS] 정치적으로 편향된 평론한은 분은 별로... [SEP]',
 '[CLS] 적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데... [SEP]',
 '[CLS] "안서는 아재들 풀발기 ㅋㄲㅋ" [SEP]',
 '[CLS] 우와 ㅋ 능력자 [SEP]',
 '[CLS] 맛녀석 콩트보다 약했음맛녀석 애청자로써 70%실력발휘 [SEP]',
 '[CLS] 주영훈 솔직히 호감임 잉꼬부부로 소문났잖아 [SEP]',
 '[CLS] 이게주간아이돌이랑머가달라... [SEP]',
 '[CLS] 아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사람 짜증나게하냐 외국서 편히살려면 아닥하고 살아라 대한민국서 취미로 돈벌어가지말고 좀 끄지라고! [SEP]',
 '[CLS] "문재인 하는게 뭐 별거있냐?ㅂㅅㅅㅋ가 하는짓인데 어련하겠어.ㅋㅋㅋ" [SEP]',
 '[CLS] 공통점은 한국남자? [SEP]',
 '[CLS] 이재진은 그게문제가아니자나 지금 ㅋㅋㅋㅋ [SEP]',
 '[CLS] "야이 OOO 색햐. 주체사상 황장엽 국립묘지에 십어넣은 이명박은 어쩔래 드으응OOO아." [SEP]',
 '[CLS] 소지섭 걸어다니는 인간 수면제나피디가 편집을 잘하겠지만걱정된다.ㅠ.ㅠ [SEP]',
 '[CLS] 원중씨........... 잘가요........ 잘살아요.......... [SEP]',
 '[CLS] 50어 애나서 언제 길러요? [SEP]',
 '[CLS] 정신나간년들 나라 돌아가는건 1도 모르지 [SEP]',
 '[CLS] "중국은 무슨 암덩어리 같음 어떻게 저렇게 세계각지에 골고루 민폐만 끼치냐" [SEP]',
 '[CLS] 알바풀었나 많이본뉴스 이프로그램기사 댓글하고 반응이 180도 다르네 ㅋㅋㅋ 의미없는 찬양질 왜케많냐 여긴 [SEP]',
 '[CLS] 택이 아부지 카리스마 뿜뿜 [SEP]',
 '[CLS] 기사에 낚이는 ㅂㅅ같은 댓글러들 [SEP]',


In [17]:
len(train)

78977

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

enc = MultiLabelBinarizer()

def multi_label(example):
  enc_label = enc.fit_transform(example['label'])
  float_arr = np.vstack(enc_label.copy()).astype(float)
  update_label = float_arr.tolist()
  return update_label

train_labels = multi_label(train)
validation_labels = multi_label(validation)
test_labels = multi_label(test)

In [25]:
test['label'][:5]

[[8], [2, 3], [2], [0], [8]]

In [22]:
test_sentences[:5]

['[CLS] 그만큼 길예르모가 잘했다고 보면되겠지 기대되네 셰이프 오브 워터 [SEP]',
 '[CLS] "1. 8넘의 문재앙" [SEP]',
 '[CLS] "문재인 정권의 내로남불은 타의 추종을 불허하네. 자한당 욕할거리도 없음." [SEP]',
 '[CLS] "짱개들 지나간 곳은 폐허된다 ㅋㅋ" [SEP]',
 '[CLS] 곱창은 자갈치~~~~~ [SEP]']

In [21]:
test_labels[:5]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [28]:
MAX_LEN = 128

def data_to_tensor(sentences, labels):
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

  attention_masks = []

  for seq in input_ids:
    seq_mask = [float(i> 0) for i in seq]
    attention_masks.append(seq_mask)
  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks

In [29]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

In [30]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [32]:
print('testset size:', len(test_labels))
print('trainset size:', len(train_labels))
print('validset size:', len(validation_labels))

testset size: 21939
trainset size: 78977
validset size: 8776


In [35]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print('Found GPU at:', device_name)
else:
  raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [36]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(f'There ard {torch.cuda.device_count()} GPU(s) available.')
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device('cpu')
  print('No GPU available, using the CPU instead')

There ard 1 GPU(s) available.
We will use the GPU: Tesla T4


In [58]:
num_labels = 9
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=num_labels, problem_type='multi_label_classification')
model.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [47]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

epochs = 1
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [48]:
def format_time(elapsed):
  elapsed_rounded = int(round((elapsed)))
  return str(datetime.timedelta(seconds=elapsed_rounded))

In [49]:
def multi_label_metrics(predictions, labels, threshold=0.5):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))

  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs >= threshold)] = 1

  y_true = labels
  accuracy = accuracy_score(y_true, y_pred)
  f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
  f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
  f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
  roc_auc = roc_auc_score(y_true, y_pred, average='micro')
  hamming = hamming_loss(y_true, y_pred)

  metrics = {'accuracy': accuracy,
            'f1_macro':f1_macro_average, 'f1_micro':f1_micro_average, 'f1_weighted':f1_weighted_average,
            'roc_auc': roc_auc, 'hamming_loss':hamming}
  return metrics

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()
for epoch_i in range(epochs):
  print()
  print('============= Epoch {:} / {:} ============='.format(epoch_i + 1, epochs))
  print('Training...')

  t0 = time.time()
  total_loss = 0

  model.train()

  for step, batch in tqdm(enumerate(train_dataloader)):
    if step % 500 == 0 and not step == 0:
      elapsed = format_time(time.time()-t0)
      print(' Batch {:>5,}  of  {:>5,}.    Elapsed: {:}'.format(step, len(train_dataloader), elapsed))
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    outputs = model(b_input_ids,
                      token_type_ids=None,
                      attention_mask=b_input_mask,
                      labels=b_labels)

    loss = outputs[0]
    total_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

    model.zero_grad()

  avg_train_loss = total_loss / len(train_dataloader)

  print()
  print('  Average training loss: {0:.4f}'.format(avg_train_loss))
  print('  Training epoch took: {:}'.format(format_time(time.time()-t0)))

print('')
print('Training complete!')


============= Epoch 1 / 1 =============
Training...


500it [05:09,  1.64it/s]

 Batch   500  of  2,469.    Elapsed: 0:05:09


1000it [10:17,  1.63it/s]

 Batch 1,000  of  2,469.    Elapsed: 0:10:18


1500it [15:26,  1.63it/s]

 Batch 1,500  of  2,469.    Elapsed: 0:15:26


1945it [19:59,  1.62it/s]

In [ ]:
print()
print('Running Validation...')

t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for batch in validation_dataloader:
  batch = tuple(t.to(device for t in batch))
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
      accum_logits.append(list(b))
    for b in label_ids:
      accum_label_ids.append(list(b))
    accum_logits = np.array(accum_logits)
    accum_label_ids = np.array(accum_label_ids)
    results = multi_label_metrics(accum_logits, accum_label_ids)

    print('Accuracy: {0:.4f}'.format(results['accuracy']))
    print('F1 (Macro) Score: {0:.4f}'.format(results['f1_macro']))
    print('F1 (Micro) Score: {0:.4f}'.format(results['f1_micro']))
    print('F1 (Weighted) Score: {0:.4f}'.format(results['f1_weighted']))
    print('ROC-AUC: {0:.4f}'.format(results['roc_auc']))
    print('Hamming Loss: {0:.4f}'.format(results['hamming_loss']))
    print('Validation took: {0:.4f}'.format(format_time(time.time()-t0)))

In [ ]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for step, batch in tqdm(enumerate(test_dataloader)):
  if step % 100 == 0 and not step == 0:
    elapsed = format_time(time.time()-t0)
    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}'.format(step, len(test_dataloader), elapsed))

  batch = tuple(t.to(device for t in batch))
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
      accum_logits.append(list(b))
    for b in label_ids:
      accum_label_ids.append(list(b))
    accum_logits = np.array(accum_logits)
    accum_label_ids = np.array(accum_label_ids)
    results = multi_label_metrics(accum_logits, accum_label_ids)

    print('Accuracy: {0:.4f}'.format(results['accuracy']))
    print('F1 (Macro) Score: {0:.4f}'.format(results['f1_macro']))
    print('F1 (Micro) Score: {0:.4f}'.format(results['f1_micro']))
    print('F1 (Weighted) Score: {0:.4f}'.format(results['f1_weighted']))
    print('ROC-AUC: {0:.4f}'.format(results['roc_auc']))
    print('Hamming Loss: {0:.4f}'.format(results['hamming_loss']))
    print('Validation took: {0:.4f}'.format(format_time(time.time()-t0)))

In [ ]:
for i in range(num_labels):
  ith_label_ids, ith_logits = [], []
  for j, labels in enumerate(accum_label_ids):
    if len(np.where(labels)[0]) == i+1:
      ith_label_ids.append(accum_label_ids[j].tolist())
      ith_logits.append(accum_logits[j].tolist())
  ith_label_ids = np.array(ith_label_ids)
  ith_logits = np.array(ith_logits)

  if len(ith_label_ids) == 0 and len(ith_logits) == 0:
    continue
  results = multi_label_metrics(ith_logits, ith_label_ids)
  accum_results.append(list(results.value()))

  print('# of labels:', i+1)
  print('Accuracy: {0:.4f}'.format(results['accuracy']))
  print('F1 (Macro) Score'.format(results['f1_macro']))
  print('F1 (Micro) Score'.format(results['f1_micro']))
  print('F1 (Weighted) Score'.format(results['f1_weighted']))
  print('ROC-AUC: {0:.4f}'.format(results['roc_auc']))
  print('Hamming Loss {0:.4f}'.format(results['hamming_loss']))

  print('\n')